### Import dependencies

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime as dt
from sklearn.metrics import confusion_matrix, classification_report

### Merge datasets

In [2]:
# Read in data
cpi = pd.read_csv("resources/cpi_final.csv")
gdp = pd.read_csv("resources/gdp_final.csv")
gdp_pct = pd.read_csv("resources/gdp_pct_chg_final.csv")
houst = pd.read_csv("resources/housing_starts_final.csv")
opg = pd.read_csv("resources/output_gap_final.csv")
rec_dt = pd.read_csv("resources/recession_dates_final.csv")
unrate = pd.read_csv("resources/unemployment_rate_final.csv")
fed_funds = pd.read_csv("resources/fed_funds_final.csv")
yield10_2 = pd.read_csv("resources/10YT_minus_2YT_final.csv")
fred = pd.read_csv("resources/FRED_data.csv")

In [3]:
# Combine all data sets into one data frame
dfs = [cpi, gdp, gdp_pct, houst, opg, rec_dt, unrate, fed_funds, yield10_2, fred]
df = reduce(lambda left,right: pd.merge(left,right,on=['quarter'],how='outer'), dfs)

In [4]:
# Sort data frame by quarter
df = df.sort_values(by=['quarter'])

In [5]:
# Drop date columns
df = df.drop(columns=['date_x','date_y'])

# Rename target column
df = df.rename(columns={'target':'recession_actual'})

In [6]:
# Set index to quarter
df = df.set_index('quarter')

In [7]:
# Save 2019 Q1 & Q2
df_2019 = df.iloc[[-4,-3],:]
df_2019 = df_2019.drop(columns=['recession_actual'])
df_2019

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,,
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,4.133333,2.401311,0.083088,0.004646,0.170000,-0.271429,4.5,1.6,1.644936,104.40334,3783.364,1.458,313000.0,4.577,14266.250
2019Q2,255.139333,21340.267,4.7,1255.666667,0.828815,3.500000,2.397813,-0.001457,0.024002,0.213333,0.254902,2.4,1.6,1.811376,103.20060,3749.471,1.457,322500.0,4.572,14511.176


In [8]:
# Drop rows with missing values
df = df.dropna()
df.tail()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,recession_actual,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,...,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,,,
2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,0.0,4.333333,1.448966,0.204683,0.083902,...,-0.113861,6.9,1.8,2.214194,104.59493,3542.412,1.451,331800.0,4.597,13728.357
2018Q2,250.578667,20510.177,7.1,1259.666667,0.589182,0.0,3.833333,1.727176,0.192007,0.075492,...,-0.251397,2.7,2.0,2.711887,103.33928,3561.592,1.461,315600.0,4.592,13939.828
2018Q3,251.828667,20749.752,4.8,1233.000000,0.821959,0.0,3.866667,1.923492,0.113663,0.047184,...,-0.432836,3.3,2.0,2.640940,103.69309,3683.981,1.462,330900.0,4.587,14114.559
2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,0.066218,...,-0.078947,2.8,1.9,2.203131,105.15026,3725.234,1.462,322800.0,4.582,14211.920
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,0.004646,...,-0.271429,4.5,1.6,1.644936,104.40334,3783.364,1.458,313000.0,4.577,14266.250


### Shift data with sliding window technique

In [9]:
df['recession_1q_out'] = df['recession_actual'].shift(-1)
df['recession_2q_out'] = df['recession_actual'].shift(-2)
df['recession_4q_out'] = df['recession_actual'].shift(-4)

In [10]:
# Create three datasets -- 1 for each model (recession 1Qtr out, 2Qtrs out, 4Qtrs out)
df_q1 = df.drop(columns=['recession_2q_out','recession_4q_out','recession_actual'])
df_q2 = df.drop(columns=['recession_4q_out','recession_1q_out','recession_actual'])
df_q4 = df.drop(columns=['recession_1q_out','recession_2q_out','recession_actual'])

In [11]:
# Delete missing values
df_q1 = df_q1.dropna()
df_q2 = df_q2.dropna()
df_q4 = df_q4.dropna()

In [12]:
# Define y variables
y1 = df_q1['recession_1q_out']
y2 = df_q2['recession_2q_out']
y3 = df_q4['recession_4q_out']

In [13]:
# Drop target
df_q1 = df_q1.drop(columns=['recession_1q_out'])
df_q2 = df_q2.drop(columns=['recession_2q_out'])
df_q4 = df_q4.drop(columns=['recession_4q_out'])

In [14]:
# Define X
X_q1 = df_q1
X_q2 = df_q2
X_q4 = df_q4

### Split and scale data

In [15]:
# Split data into training and testing (stratify=None, shuffle=False)
X1_train, X1_test, y1_train, y1_test=train_test_split(X_q1, y1, train_size=0.8, random_state=42, shuffle=False)
X2_train, X2_test, y2_train, y2_test=train_test_split(X_q2, y2, train_size=0.8, random_state=42, shuffle=False)
X3_train, X3_test, y3_train, y3_test=train_test_split(X_q4, y3, train_size=0.8, random_state=42, shuffle=False)

In [16]:
# Create scaler object
X1_scaler = StandardScaler().fit(X1_train)
X2_scaler = StandardScaler().fit(X2_train)
X3_scaler = StandardScaler().fit(X3_train)

# X full scaler
X1_full_scaler = StandardScaler().fit(X_q1)
X2_full_scaler = StandardScaler().fit(X_q2)
X3_full_scaler = StandardScaler().fit(X_q4)

In [17]:
# Scale training data
X1_train_scaled = X1_scaler.transform(X1_train)
X2_train_scaled = X2_scaler.transform(X2_train)
X3_train_scaled = X3_scaler.transform(X3_train)

# Scale testing data
X1_test_scaled = X1_scaler.transform(X1_test)
X2_test_scaled = X2_scaler.transform(X2_test)
X3_test_scaled = X3_scaler.transform(X3_test)

# Scale full X data (no splits)
X1_full_scaled = X1_full_scaler.transform(X_q1)
X2_full_scaled = X2_full_scaler.transform(X_q2)
X3_full_scaled = X3_full_scaler.transform(X_q4)

### Reshape data to fit LSTM format

In [18]:
# Method to reshape data
def reshape_data(obj):
    reshaped_obj = np.reshape(obj, (obj.shape[0], obj.shape[1], 1))
    return reshaped_obj

In [19]:
# Reshape training data
reshaped_X1_train_scaled = reshape_data(X1_train_scaled)
reshaped_X2_train_scaled = reshape_data(X2_train_scaled)
reshaped_X3_train_scaled = reshape_data(X3_train_scaled)

In [20]:
# Reshape testing data
reshaped_X1_test_scaled = reshape_data(X1_test_scaled)
reshaped_X2_test_scaled = reshape_data(X2_test_scaled)
reshaped_X3_test_scaled = reshape_data(X3_test_scaled)

In [21]:
# Reshape X_full
reshaped_X1_full = reshape_data(X1_full_scaled)
reshaped_X2_full = reshape_data(X2_full_scaled)
reshaped_X3_full = reshape_data(X3_full_scaled)

# Build Model

In [22]:
# Initialize model
model = Sequential()

In [23]:
# Add layers
model.add(LSTM(128, input_shape=(reshaped_X1_train_scaled.shape[1],1), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())  # Normalize activation outputs

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

## Train and predict on X1-Y1 data (recession 1 quarter out)

In [25]:
# Fit the model to the training data
model.fit(reshaped_X1_train_scaled, y1_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 28 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
108/108 - 14s - loss: 1.1660 - acc: 0.4352 - val_loss: 0.6789 - val_acc: 0.7500
Epoch 2/100
108/108 - 2s - loss: 0.7579 - acc: 0.5926 - val_loss: 0.6666 - val_acc: 0.7500
Epoch 3/100
108/108 - 2s - loss: 0.7314 - acc: 0.7130 - val_loss: 0.6542 - val_acc: 0.7500
Epoch 4/100
108/108 - 3s - loss: 0.7282 - acc: 0.6759 - val_loss: 0.6385 - val_acc: 0.7500
Epoch 5/100
108/108 - 4s - loss: 0.6127 - acc: 0.7037 - val_loss: 0.6208 - val_acc: 0.7500
Epoch 6/100
108/108 - 2s - loss: 0.5618 - acc: 0.7778 - val_loss: 0.6052 - val_acc: 0.7500
Epoch 7/100
108/108 - 2s - loss: 0.5914 - acc: 0.7685 - val_loss: 0.5900 - val_acc: 0.7500
Epoch 8/100
108/108 - 3s - loss: 0.5128 - acc: 0.7870 - val_loss: 0.5736 - val_acc: 0.7500
Epoch 9/100
108/108 - 2s - loss: 0.5991 - acc: 0.7593 - val_loss: 0.5659 - val_acc: 0.7500
Epoch 10/100
108/108 - 2s - loss: 0.5630 

108/108 - 2s - loss: 0.1298 - acc: 0.9630 - val_loss: 1.2563 - val_acc: 0.4643
Epoch 87/100
108/108 - 2s - loss: 0.1275 - acc: 0.9444 - val_loss: 1.1990 - val_acc: 0.5357
Epoch 88/100
108/108 - 2s - loss: 0.1238 - acc: 0.9537 - val_loss: 1.1152 - val_acc: 0.5000
Epoch 89/100
108/108 - 2s - loss: 0.1597 - acc: 0.9259 - val_loss: 1.1447 - val_acc: 0.5000
Epoch 90/100
108/108 - 2s - loss: 0.1263 - acc: 0.9630 - val_loss: 1.2465 - val_acc: 0.3929
Epoch 91/100
108/108 - 2s - loss: 0.0986 - acc: 0.9722 - val_loss: 1.3762 - val_acc: 0.3929
Epoch 92/100
108/108 - 2s - loss: 0.1101 - acc: 0.9630 - val_loss: 1.3460 - val_acc: 0.4286
Epoch 93/100
108/108 - 2s - loss: 0.1352 - acc: 0.9352 - val_loss: 1.2818 - val_acc: 0.5000
Epoch 94/100
108/108 - 2s - loss: 0.1274 - acc: 0.9537 - val_loss: 1.1500 - val_acc: 0.4643
Epoch 95/100
108/108 - 2s - loss: 0.0923 - acc: 0.9722 - val_loss: 0.9608 - val_acc: 0.5357
Epoch 96/100
108/108 - 2s - loss: 0.1110 - acc: 0.9537 - val_loss: 0.9213 - val_acc: 0.5357
E

In [26]:
# Evaluate model using test data
model_loss1, model_accuracy1 = model.evaluate(reshaped_X1_test_scaled, y1_test, verbose=2)

34/34 - 0s - loss: 0.2604 - acc: 0.9412


In [27]:
# Make predictions using test data
predictions1 = model.predict_classes(reshaped_X1_test_scaled)

In [28]:
# Compare results
one_qtr_out = pd.DataFrame({"Predicted":predictions1, "Actual":y1_test})
# one_qtr_out

#### Confusion Matrix on X1-Y1 data (recession 1 quarter out)

In [29]:
# Create confusion matrix on X1 model
con_mat = confusion_matrix(y1_test, predictions1)
print(con_mat)

[[32  2]
 [ 0  0]]


In [30]:
# Score model
print(classification_report(y1_test, predictions1))

              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97        34
         1.0       0.00      0.00      0.00         0

    accuracy                           0.94        34
   macro avg       0.50      0.47      0.48        34
weighted avg       1.00      0.94      0.97        34



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [31]:
# Save model
name1 = f"unshuffled-1q-out-{dt.datetime.now()}"
model.save(f"models/{name1}.h5")

### Predict on 2019

In [32]:
# Scale 2019 data 
scaled_X1_2019 = X1_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X1_2019 = reshape_data(scaled_X1_2019)

# Predict on 2019
pred_X1_2019 = model.predict_classes(reshaped_X1_2019)
pred_X1_2019

array([0, 0])

#### Predict on full X1

In [33]:
pred_X1_full = model.predict_classes(reshaped_X1_full)

# Preview results
X1_full_results = pd.DataFrame({"Predicted":pred_X1_full, "Actual":y1})
# X1_full_results.loc[X1_full_results["Actual"]==1]

# Export results for graphing
X1_full_results.to_csv(f"resources/predictions/X1_NS_VS20_{dt.datetime.now()}.csv")

## Train and predict on X2-Y2 data (recession 2 quarters out)

In [34]:
# Fit the model to the training data
model.fit(reshaped_X2_train_scaled, y2_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 27 samples
Epoch 1/100
108/108 - 2s - loss: 0.3953 - acc: 0.8889 - val_loss: 1.3952 - val_acc: 0.4074
Epoch 2/100
108/108 - 2s - loss: 0.2284 - acc: 0.9167 - val_loss: 1.1751 - val_acc: 0.4815
Epoch 3/100
108/108 - 2s - loss: 0.2066 - acc: 0.8889 - val_loss: 1.0892 - val_acc: 0.4815
Epoch 4/100
108/108 - 2s - loss: 0.1426 - acc: 0.9259 - val_loss: 1.2940 - val_acc: 0.5556
Epoch 5/100
108/108 - 2s - loss: 0.2345 - acc: 0.8981 - val_loss: 1.3114 - val_acc: 0.4815
Epoch 6/100
108/108 - 2s - loss: 0.1833 - acc: 0.9167 - val_loss: 1.2332 - val_acc: 0.4815
Epoch 7/100
108/108 - 2s - loss: 0.1619 - acc: 0.9259 - val_loss: 1.2021 - val_acc: 0.5556
Epoch 8/100
108/108 - 2s - loss: 0.1788 - acc: 0.9259 - val_loss: 1.1035 - val_acc: 0.5556
Epoch 9/100
108/108 - 2s - loss: 0.1108 - acc: 0.9537 - val_loss: 1.0505 - val_acc: 0.5926
Epoch 10/100
108/108 - 2s - loss: 0.1446 - acc: 0.9537 - val_loss: 1.1043 - val_acc: 0.5556
Epoch 11/100
108/108 - 2s - loss: 0.1593 - a

Epoch 90/100
108/108 - 2s - loss: 0.0093 - acc: 1.0000 - val_loss: 2.1848 - val_acc: 0.4074
Epoch 91/100
108/108 - 2s - loss: 0.0126 - acc: 1.0000 - val_loss: 2.1704 - val_acc: 0.4074
Epoch 92/100
108/108 - 2s - loss: 0.0094 - acc: 1.0000 - val_loss: 2.1731 - val_acc: 0.3704
Epoch 93/100
108/108 - 2s - loss: 0.0347 - acc: 0.9907 - val_loss: 2.3487 - val_acc: 0.2963
Epoch 94/100
108/108 - 2s - loss: 0.0129 - acc: 1.0000 - val_loss: 2.3371 - val_acc: 0.3333
Epoch 95/100
108/108 - 2s - loss: 0.0248 - acc: 0.9907 - val_loss: 2.2191 - val_acc: 0.3333
Epoch 96/100
108/108 - 2s - loss: 0.0062 - acc: 1.0000 - val_loss: 2.1653 - val_acc: 0.2963
Epoch 97/100
108/108 - 2s - loss: 0.0221 - acc: 1.0000 - val_loss: 2.1844 - val_acc: 0.3704
Epoch 98/100
108/108 - 2s - loss: 0.0077 - acc: 1.0000 - val_loss: 2.2262 - val_acc: 0.4074
Epoch 99/100
108/108 - 2s - loss: 0.0149 - acc: 1.0000 - val_loss: 2.2758 - val_acc: 0.4074
Epoch 100/100
108/108 - 2s - loss: 0.0270 - acc: 0.9907 - val_loss: 2.3517 - val

In [35]:
# Evaluate model using test data
model_loss2, model_accuracy2 = model.evaluate(reshaped_X2_test_scaled, y2_test, verbose=2)

34/34 - 0s - loss: 1.4541 - acc: 0.3529


In [36]:
# Make predictions using test data
predictions2 = model.predict_classes(reshaped_X2_test_scaled)

In [37]:
# Compare results
two_qtrs_out = pd.DataFrame({"Predicted":predictions2, "Actual":y2_test})
# two_qtrs_out

#### Confusion Matrix on X2-Y2 data (recession 2 quarters out)

In [38]:
# Create confusion matrix on X2 model
con_mat = confusion_matrix(y2_test, predictions2)
print(con_mat)

[[12 22]
 [ 0  0]]


In [39]:
# Score model
print(classification_report(y2_test, predictions2))

              precision    recall  f1-score   support

         0.0       1.00      0.35      0.52        34
         1.0       0.00      0.00      0.00         0

    accuracy                           0.35        34
   macro avg       0.50      0.18      0.26        34
weighted avg       1.00      0.35      0.52        34



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [40]:
# Save model
name2 = f"unshuffled-2q-out-{dt.datetime.now()}"
model.save(f"models/{name2}.h5")

### Predict on 2019

In [41]:
# Scale 2019 data 
scaled_X2_2019 = X2_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X2_2019 = reshape_data(scaled_X2_2019)

# Predict on 2019
pred_X2_2019 = model.predict_classes(reshaped_X2_2019)
pred_X2_2019

array([1, 1])

#### Predict on full X2

In [42]:
pred_X2_full = model.predict_classes(reshaped_X2_full)

# Preview results
X2_full_results = pd.DataFrame({"Predicted":pred_X2_full, "Actual":y2})
# X2_full_results.loc[X2_full_results["Actual"]==1]

# Export results for graphing
X2_full_results.to_csv(f"resources/predictions/X2_NS_VS20_{dt.datetime.now()}.csv")

### Train and predict on X3-Y3 data (recession 4 quarters out)

In [43]:
# Fit the model to the training data
model.fit(reshaped_X3_train_scaled, y3_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 106 samples, validate on 27 samples
Epoch 1/100
106/106 - 2s - loss: 1.0183 - acc: 0.8396 - val_loss: 1.5934 - val_acc: 0.4815
Epoch 2/100
106/106 - 2s - loss: 0.8284 - acc: 0.8491 - val_loss: 1.6719 - val_acc: 0.4444
Epoch 3/100
106/106 - 2s - loss: 0.5277 - acc: 0.8491 - val_loss: 1.6116 - val_acc: 0.3333
Epoch 4/100
106/106 - 2s - loss: 0.2606 - acc: 0.8868 - val_loss: 1.3125 - val_acc: 0.3333
Epoch 5/100
106/106 - 2s - loss: 0.2087 - acc: 0.8868 - val_loss: 0.9745 - val_acc: 0.4815
Epoch 6/100
106/106 - 2s - loss: 0.2595 - acc: 0.8491 - val_loss: 1.0309 - val_acc: 0.4074
Epoch 7/100
106/106 - 2s - loss: 0.1843 - acc: 0.9057 - val_loss: 1.2620 - val_acc: 0.4815
Epoch 8/100
106/106 - 2s - loss: 0.1797 - acc: 0.9434 - val_loss: 1.2898 - val_acc: 0.4444
Epoch 9/100
106/106 - 2s - loss: 0.1622 - acc: 0.9434 - val_loss: 1.3142 - val_acc: 0.4444
Epoch 10/100
106/106 - 2s - loss: 0.1039 - acc: 0.9528 - val_loss: 1.5165 - val_acc: 0.4444
Epoch 11/100
106/106 - 2s - loss: 0.1333 - a

Epoch 90/100
106/106 - 2s - loss: 0.0748 - acc: 0.9717 - val_loss: 2.8415 - val_acc: 0.5556
Epoch 91/100
106/106 - 2s - loss: 0.0535 - acc: 0.9717 - val_loss: 2.4422 - val_acc: 0.5926
Epoch 92/100
106/106 - 2s - loss: 0.0257 - acc: 0.9906 - val_loss: 2.2409 - val_acc: 0.5926
Epoch 93/100
106/106 - 2s - loss: 0.0943 - acc: 0.9717 - val_loss: 2.8140 - val_acc: 0.5926
Epoch 94/100
106/106 - 2s - loss: 0.0585 - acc: 0.9811 - val_loss: 4.4151 - val_acc: 0.4074
Epoch 95/100
106/106 - 2s - loss: 0.0644 - acc: 0.9811 - val_loss: 4.6664 - val_acc: 0.4074
Epoch 96/100
106/106 - 2s - loss: 0.0472 - acc: 0.9811 - val_loss: 4.4385 - val_acc: 0.4074
Epoch 97/100
106/106 - 2s - loss: 0.0310 - acc: 0.9906 - val_loss: 4.0606 - val_acc: 0.4444
Epoch 98/100
106/106 - 2s - loss: 0.0120 - acc: 0.9906 - val_loss: 3.6932 - val_acc: 0.4074
Epoch 99/100
106/106 - 2s - loss: 0.0135 - acc: 1.0000 - val_loss: 3.4333 - val_acc: 0.4444
Epoch 100/100
106/106 - 2s - loss: 0.0231 - acc: 1.0000 - val_loss: 3.3576 - val

In [44]:
# Evaluate model using test data
model_loss3, model_accuracy3 = model.evaluate(reshaped_X3_test_scaled, y3_test, verbose=2)

34/34 - 0s - loss: 2.7273 - acc: 0.5294


In [45]:
# Make predictions using test data
predictions3 = model.predict_classes(reshaped_X3_test_scaled)

In [46]:
# Compare results
four_qtrs_out = pd.DataFrame({"Predicted":predictions3, "Actual":y3_test})
# four_qtrs_out

#### Confusion Matrix on X3-Y3 data (recession 4 quarters out)

In [47]:
# Create confusion matrix on X3 model
con_mat = confusion_matrix(y3_test, predictions3)
print(con_mat)

[[18 16]
 [ 0  0]]


In [48]:
# Score model
print(classification_report(y3_test, predictions3))

              precision    recall  f1-score   support

         0.0       1.00      0.53      0.69        34
         1.0       0.00      0.00      0.00         0

    accuracy                           0.53        34
   macro avg       0.50      0.26      0.35        34
weighted avg       1.00      0.53      0.69        34



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [49]:
# Save model
name3 = f"unshuffled-4q-out-{dt.datetime.now()}"
model.save(f"models/{name3}.h5")

### Predict on 2019

In [50]:
# Scale 2019 data 
scaled_X2_2019 = X2_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X2_2019 = reshape_data(scaled_X2_2019)

# Predict on 2019
pred_X2_2019 = model.predict_classes(reshaped_X2_2019)
pred_X2_2019

array([0, 0])

#### Predict on full X3

In [51]:
pred_X3_full = model.predict_classes(reshaped_X3_full)

# Preview results
X3_full_results = pd.DataFrame({"Predicted":pred_X3_full, "Actual":y3})
# X3_full_results.loc[X3_full_results["Actual"]==1]

# Export results for graphing
X3_full_results.to_csv(f"resources/predictions/X3_NS_VS20_{dt.datetime.now()}.csv")